In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# STRATEGY: FLATTENER
# Data + NSS primitives
df = pd.read_csv("gsw_yields_2025.csv")
cols = ["Date","BETA0","BETA1","BETA2","BETA3","TAU1","TAU2"]
df = df[cols].copy()
df["Date"] = pd.to_datetime(df["Date"])
df = df.replace(-999.99, np.nan)
# Betas to decimals )
for c in ["BETA0","BETA1","BETA2","BETA3"]:
    df[c] = df[c] / 100.0


FileNotFoundError: [Errno 2] No such file or directory: 'gsw_yields_2025.csv'

In [ ]:
df = (
    df.sort_values("Date")
      .set_index("Date")
      .loc["1983-12-30":"2025-09-05"]
      .resample("W-FRI").last()
      .dropna()
)
T2, T10 = 2.0, 10.0
dt = 7/365.0  # one week in years (continuous comp)
def nss_yield(t, b0, b1, b2, b3, tau1, tau2):
    x1 = t / tau1
    x2 = t / tau2
    term1 = (1 - np.exp(-x1)) / x1
    term2 = term1 - np.exp(-x1)
    term3 = (1 - np.exp(-x2)) / x2 - np.exp(-x2)
    return b0 + b1*term1 + b2*term2 + b3*term3
# Spot curve points used for pricing/hedging today
df["y_2"]  = df.apply(lambda r: nss_yield(T2,  r.BETA0,r.BETA1,r.BETA2,r.BETA3,r.TA
df["y_10"] = df.apply(lambda r: nss_yield(T10, r.BETA0,r.BETA1,r.BETA2,r.BETA3,r.TA
df["y_1w"] = df.apply(lambda r: nss_yield(7/365, r.BETA0,r.BETA1,r.BETA2,r.BETA3,r.
# Convenience prices and DV01s at full maturities (for opening trades)
df["P2"]      = np.exp(-df["y_2"]  * T2)
df["P10"]     = np.exp(-df["y_10"] * T10)
df["DV01_2"]  = T2  * df["P2"] * 1e-4
df["DV01_10"] = T10 * df["P10"] * 1e-4
df["hedge_ratio"] = df["DV01_10"] / df["DV01_2"]   # q2 = -hedge_ratio * q10
equity_init = 1_000_000.0
margin_rate = 0.10  # 10% capital requirement (max gross = equity/margin)
# Helper: last week's curve evaluated at a shorter maturity (for pure carry)
def y_from_prev_curve(row_prev, t_short):
    return nss_yield(t_short, row_prev.BETA0,row_prev.BETA1,row_prev.BETA2,row_prev

In [2]:
# Weekly loop
equity = equity_init
equity_prev = equity_init  # for weekly return
idx = df.index.to_list()
for i, t in enumerate(idx):
    r = df.loc[t]
    is_last = (i == len(idx) - 1)
    # Close previous week's book: positions P&L (carry + yield) + cash interest
    if i > 0:
        tprev, rprev = idx[i-1], df.loc[idx[i-1]]
        q2_prev, q10_prev = df.at[tprev,"q2"], df.at[tprev,"q10"]
        # Prices when we opened last week (full maturities, last week's yields)

NameError: name 'equity_init' is not defined

In [5]:
 P2_prev  = np.exp(-df.at[tprev,"y_2"]  * T2)
        P10_prev = np.exp(-df.at[tprev,"y_10"] * T10)
        # Carry-only prices today (use LAST WEEK's curve at shorter maturities)
        y2_prev_roll  = y_from_prev_curve(rprev, T2 - dt)
        y10_prev_roll = y_from_prev_curve(rprev, T10 - dt)
        P2_carry  = np.exp(-y2_prev_roll  * (T2 - dt))
    P10_carry = np.exp(-y10_prev_roll * (T10 - dt))
        # Actual prices today at the rolled maturities (TODAY's curve)
        y2_today_roll  = nss_yield(T2 - dt,  r.BETA0,r.BETA1,r.BETA2,r.BETA3,r.TAU1
        y10_today_roll = nss_yield(T10 - dt, r.BETA0,r.BETA1,r.BETA2,r.BETA3,r.TAU1
        P2_today  = np.exp(-y2_today_roll  * (T2 - dt))
        P10_today = np.exp(-y10_today_roll * (T10 - dt))
        # Decomposition per leg
        pnl2_carry  = q2_prev  * (P2_carry  - P2_prev)
        pnl2_yield  = q2_prev  * (P2_today  - P2_carry)
        pnl10_carry = q10_prev * (P10_carry - P10_prev)
        pnl10_yield = q10_prev * (P10_today - P10_carry)
        pnl_carry = pnl2_carry + pnl10_carry
        pnl_yield = pnl2_yield + pnl10_yield
        pnl_pos   = pnl_carry + pnl_yield
        # Cash interest over the week (using LAST WEEK's cash and 1w rate)
        cash_prev = df.at[tprev,"cash"]
        y1w_prev  = df.at[tprev,"y_1w"]
        cash_int  = cash_prev * (np.exp(y1w_prev * dt) - 1.0)
        # Update equity
        equity += pnl_pos + cash_int
        # Weekly return (pnl over start-of-week equity)
        df.at[t,"ret"] = (pnl_pos + cash_int) / equity_prev
        # Store
        df.at[t,"pnl2_carry"]  = pnl2_carry
        df.at[t,"pnl2_yield"]  = pnl2_yield
        df.at[t,"pnl10_carry"] = pnl10_carry
        df.at[t,"pnl10_yield"] = pnl10_yield
        df.at[t,"pnl_carry"]   = pnl_carry
        df.at[t,"pnl_yield"]   = pnl_yield
        df.at[t,"pnl"]         = pnl_pos + cash_int
        df.at[t,"cash_interest"] = cash_int
        df.at[t,"y_2p"]  = df.at[tprev,"y_2"]   # last week same-maturity yields (r
        df.at[t,"y_10p"] = df.at[tprev,"y_10"]
   else:
        # first week: no prior positions or cash accrual
        df.at[t,"pnl"] = 0.0
        df.at[t,"cash_interest"] = 0.0
        df.at[t,"equity"] = equity
        df.at[t,"ret"] = 0.0
    if is_last:
        # Final unwind:
        df.at[t,"q2"] = 0.0
        df.at[t,"q10"] = 0.0
        df.at[t,"mv2"] = 0.0
        df.at[t,"mv10"] = 0.0
        df.at[t,"gross_exposure"] = 0.0
        df.at[t,"longpos"]  = 0.0
        df.at[t,"shortpos"] = 0.0
        df.at[t,"cash"] = equity
        df.at[t,"equity"] = equity
    else:
        hedge = r.DV01_10 / r.DV01_2
        max_gross  = equity / margin_rate                 # leverage from margin ru
        gross_unit = abs(hedge)*r.P2 + r.P10              # gross for 1 unit of q10
        scale      = max_gross / gross_unit               # choose q10 = scale
        q10 = scale
        q2  = -hedge * scale
        df.at[t,"q10"] = q10
        df.at[t,"q2"]  = q2
        # mark current positions at today's full-maturity prices 
        longpos  = q10 * r.P10
        shortpos = -q2 * r.P2      # q2 is negative → positive short proceeds
        df.at[t,"longpos"]  = longpos
        df.at[t,"shortpos"] = shortpos
        df.at[t,"mv10"] = q10 * r.P10
        df.at[t,"mv2"]  = q2  * r.P2
        df.at[t,"gross_exposure"] = abs(q2)*r.P2 + abs(q10)*r.P10

 # set NEW cash after re-hedge (what accrues until next week)
        cash = shortpos - longpos + equity               
        df.at[t,"cash"]   = cash
        df.at[t,"equity"] = equity
    # advance start-of-week equity for next return calc
    equity_prev = equity


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [6]:
# Cumulative P&L & returns
df["pnl_positions"] = (df["pnl_carry"].fillna(0) + df["pnl_yield"].fillna(0))
df["cum_pnl_positions"] = df["pnl_positions"].cumsum()
df["cum_cash_interest"] = df["cash_interest"].fillna(0).cumsum()
df["cum_pnl"] = df["pnl"].fillna(0).cumsum()          # total cumulative P&L (posit
df["cum_return"] = (1.0 + df["ret"].fillna(0)).cumprod() - 1.0
# Cumulative return plot (positions + cash)
ax = df["cum_return"].plot(title="Cumulative Return of DV01-Neutral 2s10s Strategy"
ax.set_ylabel("Cumulative return")

SyntaxError: '(' was never closed (1145029863.py, line 8)

In [ ]:
# Convexity risk time series (per +10bp parallel shift)
bp_shock = 10e-4          
face_10  = 1_000_000.0    
# 10 bps = 0.0010 in decimals
# $1mm face in the 10-year leg
# DV02 at constant maturity for zero-coupon (reuse existing P2/P10)
DV02_2  = (T2**2)  * df["P2"]
DV02_10 = (T10**2) * df["P10"]
# DV01-neutral hedge notionals (2Y face needed per $1 of 10Y face)
q10_face = face_10
q2_face  = - q10_face * df["hedge_ratio"]   # = - face_10 * (DV01_10/DV01_2)
# Weekly convexity P&L for +10bp parallel shift:
# ΔP_convex ≈ 0.5 * ( q10*DV02_10 + q2*DV02_2 ) * (Δy)^2
df["convexity_pnl_+10bp"] = 0.5 * (q10_face * DV02_10 + q2_face * DV02_2) * (bp_sho
# Plot
ax = df["convexity_pnl_+10bp"].plot(figsize=(10,4),
title="Convexity Risk (DV01-Neutral 2s10s), +10
ax.set_ylabel("ΔP from convexity (USD) for +10bp")

In [ ]:
# init first row
t0 = df.index[0]
for c in ["pnl_spread","pnl_convex","pnl_time","pnl_resid"]:
    df.at[t0, c] = 0.0
for i, t in enumerate(df.index[1:], start=1):
    tprev = df.index[i-1]
    rprev, r = df.loc[tprev], df.loc[t]
    # positions held from last week
    q2_prev, q10_prev = df.at[tprev, "q2"], df.at[tprev, "q10"]
    cash_prev, y1w_prev = df.at[tprev,"cash"], df.at[tprev,"y_1w"]
    # 1) constant-maturity yields and prices 
    y2_prev, y10_prev   = rprev.y_2, rprev.y_10
    y2_today, y10_today = r.y_2,    r.y_10
    P2_prev,  P10_prev  = df.at[tprev,"P2"],  df.at[tprev,"P10"]
    # 2) duration / convexity terms at previous curve
    DV01_2_prev  = df.at[tprev,"DV01_2"]           # $ per 1bp per $1 face
    DV01_10_prev = df.at[tprev,"DV01_10"]
    DV02_2_prev  = (T2**2)  * P2_prev  / 1e8       # $ per (bp)^2 per $1 face
    DV02_10_prev = (T10**2) * P10_prev / 1e8
    dy2, dy10 = y2_today - y2_prev, y10_today - y10_prev
    # 3) first-order SPREAD component (bp scaling retained)
    pnl_spread = -( q10_prev * DV01_10_prev * dy10*1e4
                    + q2_prev  * DV01_2_prev  * dy2*1e4 )
    # 4) second-order CONVEXITY component 
    pnl_convex = 0.5 * ( q10_prev * DV02_10_prev * (dy10*1e4)**2
                       + q2_prev  * DV02_2_prev  * (dy2*1e4)**2 )

    # 5) TIME = carry roll (using last week's curve) + cash interest
    y2_prev_roll  = y_from_prev_curve(rprev, T2  - dt)
    y10_prev_roll = y_from_prev_curve(rprev, T10 - dt)
    P2_carry  = np.exp(-y2_prev_roll  * (T2  - dt))
    P10_carry = np.exp(-y10_prev_roll * (T10 - dt))
    pnl_carry_time = q2_prev*(P2_carry - P2_prev) + q10_prev*(P10_carry - P10_prev)
    cash_int = cash_prev * (np.exp(y1w_prev*dt) - 1.0)
    pnl_time = pnl_carry_time + cash_int
    # 6) residual = actual - (spread + convex + time)
    pnl_total = df.at[t,"pnl"]  
    pnl_resid = pnl_total - (pnl_spread + pnl_convex + pnl_time)
    # store
    df.at[t,"pnl_spread"] = pnl_spread
    df.at[t,"pnl_convex"] = pnl_convex
    df.at[t,"pnl_time"]   = pnl_time
    df.at[t,"pnl_resid"]  = pnl_resid
# cumulative versions
for c in ["pnl","pnl_spread","pnl_convex","pnl_time","pnl_resid"]:

df[f"cum_{c}"] = df[c].fillna(0).cumsum()
plt.figure(figsize=(10,6))
plt.plot(df.index, df["cum_pnl"], label="Total")
plt.plot(df.index, df["cum_pnl_spread"], label="Spread")
plt.plot(df.index, df["cum_pnl_convex"], label="Convexity")
plt.plot(df.index, df["cum_pnl_time"], label="Time (carry + cash)")
plt.plot(df.index, df["cum_pnl_resid"], label="Residual")
plt.title("Cumulative Return Decomposition (DV01-Neutral 2s10s)")
plt.ylabel("Cumulative P&L ($)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 2% margin vs 10% margin (cumulative return comparison)
def run_margin(df0, margin):
df = df0.copy()
for c in ["q2","q10","cash","ret","cash_interest"]:

        df[c] = np.nan
    equity = equity_init
    equity_prev = equity_init
    idx = df.index.to_list()
    for i, t in enumerate(idx):
        r = df.loc[t]
        if i > 0:
            tprev = idx[i-1]
            rprev = df.loc[tprev]
            q2_prev  = df.at[tprev, "q2"]
            q10_prev = df.at[tprev, "q10"]
            # Previous-week constant-maturity prices
            P2_prev  = df.at[tprev, "P2"]
            P10_prev = df.at[tprev, "P10"]
            # Carry-only prices today from LAST WEEK's curve at shorter maturities
            y2_prev_roll  = y_from_prev_curve(rprev, T2  - dt)
            y10_prev_roll = y_from_prev_curve(rprev, T10 - dt)
            P2_carry  = np.exp(-y2_prev_roll  * (T2  - dt))
            P10_carry = np.exp(-y10_prev_roll * (T10 - dt))
            # Actual rolled-maturity prices today 
            y2_today_roll  = nss_yield(T2  - dt, r.BETA0,r.BETA1,r.BETA2,r.BETA3,r.
            y10_today_roll = nss_yield(T10 - dt, r.BETA0,r.BETA1,r.BETA2,r.BETA3,r.
            P2_today  = np.exp(-y2_today_roll  * (T2  - dt))
            P10_today = np.exp(-y10_today_roll * (T10 - dt))
            # Decomposition P&L
            pnl_carry = q2_prev*(P2_carry - P2_prev) + q10_prev*(P10_carry - P10_pr
            pnl_yield = q2_prev*(P2_today - P2_carry) + q10_prev*(P10_today - P10_c
            # Cash interest using LAST WEEK's cash and 1w rate
            cash_prev = df.at[tprev, "cash"]
            r1w_prev  = df.at[tprev, "y_1w"]
            cash_int  = cash_prev * (np.exp(r1w_prev * dt) - 1.0)
            pnl_total = pnl_carry + pnl_yield + cash_int
            equity   += pnl_total
            df.at[t, "ret"] = pnl_total / equity_prev
            df.at[t, "cash_interest"] = cash_int
        else:
            # first row: no prior positions or cash accrual
            df.at[t, "ret"]  = 0.0
            df.at[t, "cash"] = equity
        # Re-hedge (or final unwind)
        if i == len(idx) - 1:
            # Final unwind: 
            df.at[t, "q2"] = 0.0
            df.at[t, "q10"] = 0.0
df.at[t, "cash"] = equity
else:
h = r.hedge_ratio                              
gross_unit = abs(h) * r.P2 + r.P10
q10 = (equity / margin) / gross_unit
q2  = -h * q10
# DV01_10 / DV01_2
df.at[t, "q10"] = q10
df.at[t, "q2"]  = q2
# self-financing cash after opening the new book
df.at[t, "cash"] = equity - (q2 * r.P2 + q10 * r.P10)
equity_prev = equity
return (1.0 + df["ret"].fillna(0)).cumprod() - 1.0
cum10 = run_margin(df, 0.10)
cum02 = run_margin(df, 0.02)
ax = cum10.plot(figsize=(9,4), label="10% margin")
cum02.plot(ax=ax, label="2% margin")
ax.set_title("Cumulative Return: 2% vs 10% Margin (DV01-Neutral 2s10s)")
ax.set_ylabel("Cumulative return")